In [2]:
!pip install requests beautifulsoup4 lxml

In [107]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import warnings
warnings.filterwarnings("ignore")

# 크롤링할 사이트 주소를 정의
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행
res = requests.get(source_url)
soup = BeautifulSoup(res.content, 'lxml')
# print(soup)

contents_table = soup.find("table",{"class":""})
table_body = contents_table.find("tbody")
table_rows = table_body.find_all("tr")

# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성
page_url_base = 'https://namu.wiki'
page_urls = []
for idx in range(len(table_rows)):
    first_td = table_rows[idx].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0 :
        page_url = page_url_base + td_url[0].get('href')
        page_urls.append(page_url)

# 중복 url을 제거한다.
page_url = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

https://namu.wiki/w/%EB%8B%8C%EC%9E%90%EA%B3%A0:%20%EC%8A%A4%ED%95%80%EC%A7%93%EC%A3%BC%20%EB%A7%88%EC%8A%A4%ED%84%B0
https://namu.wiki/w/%ED%94%84%EB%A6%AC%20%EB%B9%84%EC%A6%90%EB%9D%BC
https://namu.wiki/w/%EB%B6%84%EB%A5%98:%EC%8A%88%ED%8D%BC%20%ED%9E%88%EC%96%B4%EB%A1%9C%20%EB%8C%80%EC%A0%84%20%EC%8B%9C%EB%A6%AC%EC%A6%88
https://namu.wiki/w/%EC%82%AC%EC%9D%B4%ED%82%A5%20%EB%B9%84%EC%BB%A8
https://namu.wiki/w/%EB%8B%8C%EC%9E%90%EA%B3%A0(%EB%8B%8C%EC%9E%90%EA%B3%A0)


In [108]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비
columns = ['title', 'contents_clearfix']
df= pd.DataFrame(columns=columns)

for page_url in page_urls:
    req = requests.get(page_url)
    soup = BeautifulSoup(req.content, 'lxml')
    title = soup.find("h1",class_ = "")
    contents_clearfix = soup.find("div", class_="w")

    # 페이지 내 제목 정보에서 개행 문자를 제거한뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n",' ')
    else:
        row_title = ""
    
    # 페이지 정보에서 개행 문자를 제거한뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if contents_clearfix is not None:
        row_contents_clearfix = contents_clearfix.text.replace('\n',' ')
    else:
        row_contents_clearfix = ""

        
    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row = [row_title, row_contents_clearfix]
    series = pd.Series(row, index=df.columns)
    df= df.append(series, ignore_index=True)

In [109]:
df.head(5)

,title,contents_clearfix
0,닌자고: 스핀짓주 마스터,[ 시리즈 목록 ] 제목파일럿 시즌Pilot Episodes→닌자고 Rise o...
1,프리 비즐라,시리즈의 주요 빌런 [ 펼치기 · 접기 ]은하 제국다스 시디어스 (쉬브 팰퍼틴 황...
2,분류:슈퍼 히어로 대전 시리즈,이 분류에 대한 설명은 슈퍼 히어로 대전 시리즈 문서나 문서를 참고하십시오.
3,사이킥 비컨,Psychic Beacon1. 개요2. 미션에서의 등장2.1. 연합군2.2. 소련군...
4,닌자고(닌자고),[ 시리즈 목록 ] 제목파일럿 시즌Pilot Episodes→닌자고 Rise o...


# <Step2.추출> : 키워드 추출
## [텍스트 데이터 전처리]

In [112]:
import re
def text_cleaning(text):
    hangul = re.compile('[^ㄱ-ㅣ 가-힣]+')# 한글의 정규표현식을 나타냅니다.
    result = hangul.sub('',text)
    return result

In [113]:
print(text_cleaning(df['contents_clearfix'][0]))

  시리즈 목록   제목파일럿 시즌 닌자고    그린 닌자고  골드 닌자고     리부티드마스코트카이그린 닌자로이드 쟌제목티타늄 닌자고    고스트 닌자고 닌자고 대 스카이 해적 유령의 날   마스코트카이로이드 제이콜제목타임블레이드   마스터즈  드래곤 헌터도깨비 습격   마스코트카이 니야가마돈우콜  제목불의 닌자 대 얼음의 황제    게임 속으로 레벨 업 스컬 던전 속으로   미지의 섬 마스코트카이제이콜 로이드제목바다마스코트니야닌자고 스토리 진행스핀짓주 마스터닌자고  제작 시나리오헤이그먼 형제방영시기년 월 일 개요 소개 등장인물 에피소드 목록 미니무비 제품 목록 메인 제품 스핀짓주 제품  갤러리 여담 개요편집닌자고의 기념비적인 첫 에피소드 다만 프롤로그 격인 작품이라 메인 에피소드에는 들어가지 않으며 시즌 으로 분류된다타이틀 캐릭터는 카이 등장하는 세력은 해골군단 소개편집시간이라는 개념도 없던 머나먼 옛적에 최초의 스핀짓주 마스터가 네 개의 스핀짓주 황금 무기를 이용해 닌자고 세계를 창조했어요 황금 무기는 너무나도 강력하여 누구도 한꺼번에 네 개를 모두 다룰 수 없었죠 그는 두 아들에게 무기를 수호할 것을 부탁하고 세상을 떴어요 그런데 그만 형인 제왕 가마돈이 어둠의 힘에 사로잡혀 무기를 차지하고 싶다는 마음을 먹게 되어요 결국 형제는 서로 적이 되어 싸움을 벌였고 최종 결전에서 패한 제왕 가마돈은 지하 세계로 쫓겨났죠평화롭던 온 세상이 위기에 빠졌다 사악한 제왕 가마돈이 강력한 해골 군단을 앞세워 세상을 나쁘게 변화시키려 하는데 제왕 가마돈에 맞서 싸울 수 있는 사람은 오직 마스터 우와 그가 훈련 중인 네명의 닌자 견습생 제이 카이 쟌 콜 뿐 번개의 쌍절곤을 가진 번개의 블루닌자 제이 가마돈에게 잡혀간 동생 니야를 구하기 위해 닌자가 된 레드닌자 카이 얼음의 수리검을 가진 얼음의 화이트 닌자 쟌 흙의 블랙닌자로 진동의 낫이 무기인 팀의 리더 콜 과연 이들이 가진 개의 황금무기를 지켜 악당 가마돈으로부터 위기에 처한 이 세상을 구해낼 수 있을까카이가 마스터 

In [114]:
# 각 피처마다 데이터 전처리를 적용합니다.
df['title'] =df['title'].apply(lambda x : text_cleaning(x))
df['contents_clearfix'] =df['contents_clearfix'].apply(lambda x : text_cleaning(x))
df.head(5)


,title,contents_clearfix
0,닌자고 스핀짓주 마스터,시리즈 목록 제목파일럿 시즌 닌자고 그린 닌자고 골드 닌자고 ...
1,프리 비즐라,시리즈의 주요 빌런 펼치기 접기 은하 제국다스 시디어스 쉬브 팰퍼틴 황제다스 ...
2,분류슈퍼 히어로 대전 시리즈,이 분류에 대한 설명은 슈퍼 히어로 대전 시리즈 문서나 문서를 참고하십시오
3,사이킥 비컨,개요 미션에서의 등장 연합군 소련군 유리의 복수 효과 유사품 개요편집커맨드 앤 ...
4,닌자고닌자고,시리즈 목록 제목파일럿 시즌 닌자고 그린 닌자고 골드 닌자고 ...


## [말뭉치 만들기]

In [123]:
title_corpus = "".join(df['title'].tolist())
content_corpus = "".join(df['contents_clearfix'].tolist())
print(title_corpus)

닌자고 스핀짓주 마스터 프리 비즐라 분류슈퍼 히어로 대전 시리즈 사이킥 비컨 닌자고닌자고 마블 시네마틱 유니버스평가 서울 공화국외국 사례 그린 닌자고  골드 닌자고 마이너즈 쇼타임 임파 진흥고등학교 야구부 슈퍼슬램 사이킥 비컨 하드 디스크 드라이브   진격의 거인결말 논란 서울 공화국외국 사례 남부정류장 아파트웹툰 흑의 홍차 틀 시사교양  서번트랜서우미인 서울대학교 판다 빅히트 뮤직 버즈 올드린 폴란드볼컨트리볼  리그 고사카 다이마오 리그 오브 레전드 필립 마운트배튼 세키로 섀도우 다이 트와이스의수 닌자 도구 피오 맹주 사후 리산드라 크레이지레이싱 카트라이더리그 악동 활잡이천하제일상 거상 오와라이 빅히트 뮤직 북구광주광역시 카를로스 수베로년 틴 타이탄  퓨처아이돌 셀레나크로노 아크 수첩 틀차 세계대전이탈리아 항공기 마블 시네마틱 유니버스평가 교부    제임스 딘 로열 패밀리닌자고 메가톤맨부 리본즈 건담 클레드 틀차 세계대전이탈리아 항공기 카를로스 수베로년 곽도  셀레나크로노 아크 리그 오브 레전드 파일  바이오하자드 스테이지 이범승 부엉이에듀등장인물 세키로 섀도우 다이 트와이스의수 닌자 도구 군통령 남부정류장 리그 감해국 오마이걸 생물정보학 이미테이션드라마 송석래 코야스 타케히토출연작 파일아파트 표지 우즈이 텐겐 서울 공화국외국 사례 트리니티 수첩년 상반기 세키로 섀도우 다이 트와이스의수 닌자 도구 마리오마리오 시리즈    경남대학교 전혜진작가 레베카영화   서번트랜서우미인  전혜진작가 가이코 나연 틀차 세계대전이탈리아 항공기 세키로 섀도우 다이 트와이스의수 닌자 도구 서울 공화국외국 사례     
